In [1]:
!pip install xmltodict


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from requests import Session

import threading
import random
import time
import os

import utils as ut
import html_utils as hu

try:
    ut.fu.copy_folder_structure(
        ut.song_ids_of_song_parts_dir,
        ut.song_parts_from_ids_dir
    )
except Exception as e:
    print(e)

time_to_sleep_between_song_parts = random.uniform(2, 5)

start_letter = 'a'
end_letter = 'z'

song_parts_list = []

KeyboardInterrupt: 

In [ ]:
lock = threading.Lock()

thread_per_batch = 10

thread_list = []

total_saved = 0

In [ ]:
class Downloader():
    host = "https://api.hooktheory.com/v1/songs/public/"
    song_parts_item = None
    client = None

    json_file_path = None
    song_link = None

    def __init__(self):
        self.client = Session()


        time_to_sleep = random.uniform(0, 1)
        time.sleep(time_to_sleep)
        global song_parts_list
        
        with lock:
            if len(song_parts_list) > 0:
                self.song_parts_item = song_parts_list.pop(0)

                self.song_link = self.song_parts_item["link"]
                self.json_file_path = ut.fu.song_link_to_relative_html_file_path(self.song_link)
                self.json_file_path = self.json_file_path.replace(".html", ".json")

                self.json_file_path = ut.fu.relative_to_absolute_path(
                    relative_path = self.json_file_path,
                    root_path = ut.song_parts_from_ids_dir
                )
            else:
                self.song_parts_item = None

    def download(self):
        global time_to_sleep_between_song_parts

        song_part_ids = list(self.song_parts_item["song_parts_ids"])

        song_entire_data = dict(self.song_parts_item)

        song_entire_data["song_parts"] = []
        del song_entire_data["song_parts_ids"]

        for song_part_id in song_part_ids:
            song_part_url = f"{self.host}{song_part_id}"
            response = hu.make_request(self.client, song_part_url)

            if response["status_code"] == 200:
                song_entire_data["song_parts"].append(
                    {
                        "song_part_id": song_part_id,
                        "metadata": response["metadata"],
                        "main_data_type": response["main_data_type"],
                        "main_data": response["main_data"]
                    }
                )

                time.sleep(time_to_sleep_between_song_parts)
            else:
                error_message = f"FAIL: download - status_code: {response['status_code']}, url: {song_part_url}"
                print(error_message)
                print()
                break

        n_part_downloaded = len(song_entire_data["song_parts"])
        n_part_total = len(song_part_ids)

        if n_part_downloaded == n_part_total:
            global total_saved
            with lock:
                total_saved += 1
                print(f"--------> TOTAL SAVED: {total_saved}")

            ut.fu.write_data_to_json_file(
                song_entire_data,
                self.json_file_path
            )

            print(f"SUCCESS: {self.song_link}")
            print(f"--------> SAVED: {self.json_file_path}")
            print()
        else:
            print(f"ACK: {n_part_downloaded}/{n_part_total} part(s) downloaded - {self.song_link}")
            global song_parts_list

            with lock:
                song_parts_list.append(self.song_parts_item)
            
            print(f"--------> APPENDED BACK: {self.song_link}")
            print()

    def download_song_parts(self):
        if self.song_parts_item is not None:
            # Nếu file đã tồn tại và file size > 10 KB thì skip
            if os.path.exists(self.json_file_path) and os.path.getsize(self.json_file_path) > 10000:
                print(f"SKIP: {self.json_file_path}")
                print()
            else:
                print(f"GETTING: {self.song_parts_item['link']}")
                self.download()
        else:
            print("INFO: download_song_parts - No more song to handle")
            print()
            self.environment.runner.quit()

In [ ]:
for letter_to_crawl in range(ord(start_letter), ord(end_letter) + 1):
    song_parts_list.extend(
        ut.read_song_id_of_song_parts_list(
            chr(letter_to_crawl)
        )
    )

    while len(song_parts_list) > 0:
        for i in range(thread_per_batch):
            downloader = Downloader()
            thread = threading.Thread(target=downloader.download_song_parts)
            thread_list.append(thread)
            thread.start()
            time.sleep(0.5)

        for thread in thread_list:
            thread.join()

        thread_list.clear()

        time.sleep(5)

    print()
    print(f"--------> LETTER {chr(letter_to_crawl)} DONE")
    print()